In [0]:
%pip install msal deltalake

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 MB 170.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 181.7 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
from msal import ConfidentialClientApplication
import requests
from deltalake import DeltaTable

In [0]:
kv_uri = 'https://kvfabricprodeus2rh.vault.azure.net/'
client_id_secret_name = 'fuam-spn-client-id'
tenant_id_secret_name = 'fuam-spn-tenant-id'
client_secret_name = 'fuam-spn-secret'

In [0]:
dbutils.secrets.listScopes()

[SecretScope(name='adf-pipeline'),
 SecretScope(name='akv-test'),
 SecretScope(name='databricks-kv-rh-scope'),
 SecretScope(name='kvfabricprodeus2rh')]

In [0]:
client_id = dbutils.secrets.get('kvfabricprodeus2rh', client_id_secret_name)
tenant_id = dbutils.secrets.get('kvfabricprodeus2rh', tenant_id_secret_name)
client_secret = dbutils.secrets.get('kvfabricprodeus2rh', client_secret_name)
app = ConfidentialClientApplication(
    client_id,
    client_secret,
    authority=f"https://login.microsoftonline.com/{tenant_id}"
)
result = app.acquire_token_for_client(scopes=["https://onelake.fabric.microsoft.com/.default"])
access_token = result['access_token']
headers = {
    "Authorization": f"Bearer {access_token}"
}



In [0]:
onelake_path = 'abfss://c0a7b8a9-eb12-495a-b863-2cb583e31154@onelake.dfs.fabric.microsoft.com/b029b982-e97a-4007-8223-10acbeb425d4/Tables/insider_sentiment'
dt = DeltaTable(onelake_path, storage_options={"bearer_token": f"{access_token}", "use_fabric_endpoint": "true"})



In [0]:
df = dt.toDF()

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File <command-6076388849604664>, line 1
----> 1 df = dt.toDF()

AttributeError: 'DeltaTable' object has no attribute 'toDF'

In [0]:

df = spark.read.format("delta") \
    .option("bearer_token", access_token) \
    .option("use_fabric_endpoint", "true") \
       .load(onelake_path)

display(df)

---------------------------------------------------------------------------
SparkConnectGrpcException                 Traceback (most recent call last)
File <command-6076388849604665>, line 6
      1 df = spark.read.format("delta") \
      2     .option("bearer_token", access_token) \
      3     .option("use_fabric_endpoint", "true") \
      4        .load(onelake_path)
----> 6 display(df)

File /databricks/python_shell/lib/dbruntime/display.py:142, in Display.display(self, input, *args, **kwargs)
    140 # This version is for Serverless + Spark Connect dogfooding.
    141 elif self.spark_connect_enabled and isinstance(input, ConnectDataFrame):
--> 142     self.display_connect_table(input, **kwargs)
    143 elif isinstance(input, ConnectDataFrame):
    144     if input.isStreaming:

File /databricks/python_shell/lib/dbruntime/display.py:103, in Display.display_connect_table(self, df, **kwargs)
     98 except Exception as e:
     99     raise type(
    100         e
    101     )("IPyt

In [0]:
pandas_df = dt.to_pandas()

pandas_df

,data,symbol,hash,_last_modified_dttm,_source
0,"[{'change': -5188, 'month': 2, 'mspr': -7.6170...",TSLA,7dba504b58f44fc4b601480111b0b11652873821751258...,2025-02-01 21:03:40.097701+00:00,finnhubb
1,"[{'change': -5188, 'month': 2, 'mspr': -7.6170...",TSLA,7dba504b58f44fc4b601480111b0b11652873821751258...,2025-01-31 18:18:04.545987+00:00,finnhubb


In [0]:
spark_df = spark.createDataFrame(pandas_df)

display(spark_df)

data,symbol,hash,_last_modified_dttm,_source
"List(List(-5188, 2, -7.61709, TSLA, 2015), List(0, 3, 0.0, TSLA, 2015), List(0, 4, 0.0, TSLA, 2015), List(33333, 5, 28.826048, TSLA, 2015), List(0, 6, 0.0, TSLA, 2015), List(532000, 1, 100.0, TSLA, 2016), List(783916, 2, 100.0, TSLA, 2016), List(4113, 3, 10.680065, TSLA, 2016), List(2721908, 11, 97.408394, TSLA, 2016), List(-161815, 12, -72.120674, TSLA, 2016), List(-1000, 1, -5.8139534, TSLA, 2017), List(-158206, 2, -65.954895, TSLA, 2017), List(98613, 3, 91.278755, TSLA, 2017), List(-2190, 7, -30.458971, TSLA, 2017), List(-1889, 8, -2.0375144, TSLA, 2017), List(527, 9, 4.6526, TSLA, 2017), List(105, 11, 100.0, TSLA, 2017), List(-3190, 1, -37.344883, TSLA, 2018), List(-1000, 2, -55.493896, TSLA, 2018), List(7520, 3, 30.246964, TSLA, 2018), List(0, 4, 0.0, TSLA, 2018), List(54332, 5, 100.0, TSLA, 2018), List(72240, 6, 86.31237, TSLA, 2018), List(0, 7, 0.0, TSLA, 2018), List(1337, 9, 33.466835, TSLA, 2018), List(34879, 10, 85.84755, TSLA, 2018), List(33215, 11, 28.002361, TSLA, 2018), List(-788, 12, -1.1592668, TSLA, 2018), List(-2380, 1, -3.574647, TSLA, 2019), List(-2380, 2, -2.2640362, TSLA, 2019), List(-304, 3, -0.2867059, TSLA, 2019), List(-516, 10, -2.5151103, TSLA, 2019), List(-14816, 11, -9.653249, TSLA, 2019), List(-21284, 12, -60.55881, TSLA, 2019), List(-250, 4, -4.8076925, TSLA, 2020), List(-95, 5, -1.6622922, TSLA, 2020), List(-1801, 6, -6.79597, TSLA, 2020), List(-420, 7, -0.56919825, TSLA, 2020), List(-255, 8, -0.80075365, TSLA, 2020), List(-12315, 9, -9.00779, TSLA, 2020), List(-10549, 10, -2.0564194, TSLA, 2020), List(-1740, 11, -1.5106789, TSLA, 2020), List(-35302, 12, -17.810223, TSLA, 2020), List(-2113, 1, -6.5391636, TSLA, 2021), List(-31250, 2, -8.662653, TSLA, 2021), List(5540, 3, 12.209097, TSLA, 2021), List(-2113, 4, -1.1655166, TSLA, 2021), List(-1750, 5, -7.368421, TSLA, 2021), List(4261, 6, 20.06404, TSLA, 2021), List(-6209, 7, -37.885166, TSLA, 2021), List(-43000, 8, -33.72549, TSLA, 2021), List(2905, 9, 3.443371, TSLA, 2021), List(-212679, 10, -21.95557, TSLA, 2021), List(-9303898, 11, -35.160435, TSLA, 2021), List(-6548546, 12, -18.577404, TSLA, 2021), List(-11750, 1, -35.87786, TSLA, 2022), List(-29750, 2, -34.29395, TSLA, 2022), List(-33321, 3, -29.829996, TSLA, 2022), List(-9655500, 4, -99.78298, TSLA, 2022), List(-33400, 5, -33.266933, TSLA, 2022), List(-32991, 6, -28.806307, TSLA, 2022), List(-21000, 7, -18.58407, TSLA, 2022), List(-9443720, 8, -99.59398, TSLA, 2022), List(-951083, 9, -83.984474, TSLA, 2022), List(-14250, 10, -40.425533, TSLA, 2022), List(-21534251, 11, -99.90257, TSLA, 2022), List(-30121914, 12, -99.57006, TSLA, 2022))",TSLA,7dba504b58f44fc4b601480111b0b116528738217512585f5c9677e03bcc4b9f,2025-02-01T21:03:40.097Z,finnhubb
"List(List(-5188, 2, -7.61709, TSLA, 2015), List(0, 3, 0.0, TSLA, 2015), List(0, 4, 0.0, TSLA, 2015), List(33333, 5, 28.826048, TSLA, 2015), List(0, 6, 0.0, TSLA, 2015), List(532000, 1, 100.0, TSLA, 2016), List(783916, 2, 100.0, TSLA, 2016), List(4113, 3, 10.680065, TSLA, 2016), List(2721908, 11, 97.408394, TSLA, 2016), List(-161815, 12, -72.120674, TSLA, 2016), List(-1000, 1, -5.8139534, TSLA, 2017), List(-158206, 2, -65.954895, TSLA, 2017), List(98613, 3, 91.278755, TSLA, 2017), List(-2190, 7, -30.458971, TSLA, 2017), List(-1889, 8, -2.0375144, TSLA, 2017), List(527, 9, 4.6526, TSLA, 2017), List(105, 11, 100.0, TSLA, 2017), List(-3190, 1, -37.344883, TSLA, 2018), List(-1000, 2, -55.493896, TSLA, 2018), List(7520, 3, 30.246964, TSLA, 2018), List(0, 4, 0.0, TSLA, 2018), List(54332, 5, 100.0, TSLA, 2018), List(72240, 6, 86.31237, TSLA, 2018), List(0, 7, 0.0, TSLA, 2018), List(1337, 9, 33.466835, TSLA, 2018), List(34879, 10, 85.84755, TSLA, 2018), List(33215, 11, 28.002361, TSLA, 2018), List(-788, 12, -1.1592668, TSLA, 2018), List(-2380, 1, -3.574647, TSLA, 2019), List(-2380, 2, -2.2640362, TSLA, 2019), List(-304, 3, -0.2867059, TSLA, 2019), List(-516, 10, -2.5151103, TSLA, 2019), List(-14816, 11, -9.653249, TSLA,

# Attempt

In [0]:

spark.conf.set("fs.azure.account.auth.type.onelake.dfs.fabric.microsoft.com", "OAuth")
spark.conf.set("fs.azure.account.oauth2.access.token", access_token)


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-6076388849604669>, line 1
----> 1 spark.conf.set("fs.azure.account.auth.type.onelake.dfs.fabric.microsoft.com", "OAuth")
      2 spark.conf.set("fs.azure.account.oauth2.access.token", access_token)

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/conf.py:46, in RuntimeConf.set(self, key, value)
     44 op_set = proto.ConfigRequest.Set(pairs=[proto.KeyValue(key=key, value=value)])
     45 operation = proto.ConfigRequest.Operation(set=op_set)
---> 46 result = self._client.config(operation)
     47 for warn in result.warnings:
     48     warnings.warn(warn)

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/client/core.py:1867, in SparkConnectClient.config(self, operation)
   1865     raise SparkConnectException("Invalid state during retry exception handling.")
   1866 except E

In [0]:
dir(dt)

['__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__datafusion_table_provider__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_storage_options',
 '_stringify_partition_values',
 '_table',
 'alter',
 'cleanup_metadata',
 'count',
 'create',
 'create_checkpoint',
 'create_write_transaction',
 'delete',
 'file_uris',
 'files',
 'files_by_partitions',
 'get_add_actions',
 'history',
 'is_deltatable',
 'load_as_version',
 'load_cdf',
 'merge',
 'metadata',
 'optimize',
 'partitions',
 'protocol',
 'repair',
 'restore',
 'schema',
 'table_config',
 'table_uri',
 'to_pandas',
 'to_pyarrow_dataset',
 'to_pyarrow_table',
 'transaction_

In [0]:
df = spark.read.format('delta').load('abfss://c0a7b8a9-eb12-495a-b863-2cb583e31154@onelake.dfs.fabric.microsoft.com/b029b982-e97a-4007-8223-10acbeb425d4/Tables/insider_sentiment')

display(df)

---------------------------------------------------------------------------
SparkConnectGrpcException                 Traceback (most recent call last)
File <command-8192290817496715>, line 3
      1 df = spark.read.format('delta').load('abfss://c0a7b8a9-eb12-495a-b863-2cb583e31154@onelake.dfs.fabric.microsoft.com/b029b982-e97a-4007-8223-10acbeb425d4/Tables/insider_sentiment')
----> 3 display(df)

File /databricks/python_shell/lib/dbruntime/display.py:142, in Display.display(self, input, *args, **kwargs)
    140 # This version is for Serverless + Spark Connect dogfooding.
    141 elif self.spark_connect_enabled and isinstance(input, ConnectDataFrame):
--> 142     self.display_connect_table(input, **kwargs)
    143 elif isinstance(input, ConnectDataFrame):
    144     if input.isStreaming:

File /databricks/python_shell/lib/dbruntime/display.py:103, in Display.display_connect_table(self, df, **kwargs)
     98 except Exception as e:
     99     raise type(
    100         e
    101     )

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

df_dedup = df.withColumn(
    "rank", 
    F.row_number().over(
        Window.partitionBy("data").orderBy(F.desc("_last_modified_dttm"))
    )
).filter(F.col("rank") == 1).drop("rank")

display(df_dedup)

data,symbol,hash,_last_modified_dttm,_source
"List(List(-5188, 2, -7.61709, TSLA, 2015), List(0, 3, 0.0, TSLA, 2015), List(0, 4, 0.0, TSLA, 2015), List(33333, 5, 28.826048, TSLA, 2015), List(0, 6, 0.0, TSLA, 2015), List(532000, 1, 100.0, TSLA, 2016), List(783916, 2, 100.0, TSLA, 2016), List(4113, 3, 10.680065, TSLA, 2016), List(2721908, 11, 97.408394, TSLA, 2016), List(-161815, 12, -72.120674, TSLA, 2016), List(-1000, 1, -5.8139534, TSLA, 2017), List(-158206, 2, -65.954895, TSLA, 2017), List(98613, 3, 91.278755, TSLA, 2017), List(-2190, 7, -30.458971, TSLA, 2017), List(-1889, 8, -2.0375144, TSLA, 2017), List(527, 9, 4.6526, TSLA, 2017), List(105, 11, 100.0, TSLA, 2017), List(-3190, 1, -37.344883, TSLA, 2018), List(-1000, 2, -55.493896, TSLA, 2018), List(7520, 3, 30.246964, TSLA, 2018), List(0, 4, 0.0, TSLA, 2018), List(54332, 5, 100.0, TSLA, 2018), List(72240, 6, 86.31237, TSLA, 2018), List(0, 7, 0.0, TSLA, 2018), List(1337, 9, 33.466835, TSLA, 2018), List(34879, 10, 85.84755, TSLA, 2018), List(33215, 11, 28.002361, TSLA, 2018), List(-788, 12, -1.1592668, TSLA, 2018), List(-2380, 1, -3.574647, TSLA, 2019), List(-2380, 2, -2.2640362, TSLA, 2019), List(-304, 3, -0.2867059, TSLA, 2019), List(-516, 10, -2.5151103, TSLA, 2019), List(-14816, 11, -9.653249, TSLA, 2019), List(-21284, 12, -60.55881, TSLA, 2019), List(-250, 4, -4.8076925, TSLA, 2020), List(-95, 5, -1.6622922, TSLA, 2020), List(-1801, 6, -6.79597, TSLA, 2020), List(-420, 7, -0.56919825, TSLA, 2020), List(-255, 8, -0.80075365, TSLA, 2020), List(-12315, 9, -9.00779, TSLA, 2020), List(-10549, 10, -2.0564194, TSLA, 2020), List(-1740, 11, -1.5106789, TSLA, 2020), List(-35302, 12, -17.810223, TSLA, 2020), List(-2113, 1, -6.5391636, TSLA, 2021), List(-31250, 2, -8.662653, TSLA, 2021), List(5540, 3, 12.209097, TSLA, 2021), List(-2113, 4, -1.1655166, TSLA, 2021), List(-1750, 5, -7.368421, TSLA, 2021), List(4261, 6, 20.06404, TSLA, 2021), List(-6209, 7, -37.885166, TSLA, 2021), List(-43000, 8, -33.72549, TSLA, 2021), List(2905, 9, 3.443371, TSLA, 2021), List(-212679, 10, -21.95557, TSLA, 2021), List(-9303898, 11, -35.160435, TSLA, 2021), List(-6548546, 12, -18.577404, TSLA, 2021), List(-11750, 1, -35.87786, TSLA, 2022), List(-29750, 2, -34.29395, TSLA, 2022), List(-33321, 3, -29.829996, TSLA, 2022), List(-9655500, 4, -99.78298, TSLA, 2022), List(-33400, 5, -33.266933, TSLA, 2022), List(-32991, 6, -28.806307, TSLA, 2022), List(-21000, 7, -18.58407, TSLA, 2022), List(-9443720, 8, -99.59398, TSLA, 2022), List(-951083, 9, -83.984474, TSLA, 2022), List(-14250, 10, -40.425533, TSLA, 2022), List(-21534251, 11, -99.90257, TSLA, 2022), List(-30121914, 12, -99.57006, TSLA, 2022))",TSLA,7dba504b58f44fc4b601480111b0b116528738217512585f5c9677e03bcc4b9f,2025-02-01T21:03:40.097701Z,finnhubb


In [0]:
df_exploded = df_dedup.withColumn("data", F.explode("data"))

display(df_exploded)

data,symbol,hash,_last_modified_dttm,_source
"List(-5188, 2, -7.61709, TSLA, 2015)",TSLA,7dba504b58f44fc4b601480111b0b116528738217512585f5c9677e03bcc4b9f,2025-02-01T21:03:40.097701Z,finnhubb
"List(0, 3, 0.0, TSLA, 2015)",TSLA,7dba504b58f44fc4b601480111b0b116528738217512585f5c9677e03bcc4b9f,2025-02-01T21:03:40.097701Z,finnhubb
"List(0, 4, 0.0, TSLA, 2015)",TSLA,7dba504b58f44fc4b601480111b0b116528738217512585f5c9677e03bcc4b9f,2025-02-01T21:03:40.097701Z,finnhubb
"List(33333, 5, 28.826048, TSLA, 2015)",TSLA,7dba504b58f44fc4b601480111b0b116528738217512585f5c9677e03bcc4b9f,2025-02-01T21:03:40.097701Z,finnhubb
"List(0, 6, 0.0, TSLA, 2015)",TSLA,7dba504b58f44fc4b601480111b0b116528738217512585f5c9677e03bcc4b9f,2025-02-01T21:03:40.097701Z,finnhubb
"List(532000, 1, 100.0, TSLA, 2016)",TSLA,7dba504b58f44fc4b601480111b0b116528738217512585f5c9677e03bcc4b9f,2025-02-01T21:03:40.097701Z,finnhubb
"List(783916, 2, 100.0, TSLA, 2016)",TSLA,7dba504b58f44fc4b601480111b0b116528738217512585f5c9677e03bcc4b9f,2025-02-01T21:03:40.097701Z,finnhubb
"List(4113, 3, 10.680065, TSLA, 2016)",TSLA,7dba504b58f44fc4b601480111b0b116528738217512585f5c9677e03bcc4b9f,2025-02-01T21:03:40.097701Z,finnhubb
"List(2721908, 11, 97.408394, TSLA, 2016)",TSLA,7dba504b58f44fc4b601480111b0b116528738217512585f5c9677e03bcc4b9f,2025-02-01T21:03:40.097701Z,finnhubb
"List(-161815, 12, -72.120674, TSLA, 2016)",TSLA,7dba504b58f44fc4b601480111b0b116528738217512585f5c9677e03bcc4b9f,2025-02-01T21:03:40.097701Z,finnhubb
